In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras.optimizers import SGD
from keras import regularizers

import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras import models
from keras import layers
from keras.models import Model
import matplotlib.pyplot as plt
from keras.models import Sequential

from keras.callbacks import TensorBoard

C:\Users\dell\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#import data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
 
weight_decay = 0.0005
nb_epoch=200
batch_size=128

In [3]:
from keras.models import load_model
model = load_model('0.8ent_b_cifar10vgg16_5_1.h5')
model.load_weights('0.8ent_b_cifar10vgg16_5_1_weights.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
__________

In [4]:
tensor =[]
for i in range(0,128):
   
    u=model.get_layer('conv2d_4').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [11.317330198587818, 40.18912508750436, 35.133658431381]
1 [11.368109942889792, 15.98829962167586, 14.694114534828621]
2 [23.273253092782742, 17.510516583008037, 3.803675710884699]
3 [21.952739258767686, 33.63080155478253, 19.092064965748783]
4 [6.100081602316809, 16.160523725558853, 14.702702742218024]
5 [3.6487042413281507, 8.170473828367138, 11.896150746205537]
6 [15.452362165146438, 31.669089617808154, 19.548816233842953]
7 [3.3438065069539946, 4.972126922842586, 11.81860206063575]
8 [43.47361966135982, 26.618113545039105, 10.355292665185308]
9 [19.003342685791953, 15.276291867832363, 15.350305528116323]
10 [13.643831206153008, 12.989342952156996, 12.765431219378767]
11 [6.024644030743565, 15.029895499380736, 5.346327355328902]
12 [36.47754378151908, 21.186782104573012, 25.17772241511781]
13 [17.756974716999476, 23.84407443204569, 7.271462734783472]
14 [17.468563563353428, 14.155726341010451, 16.343239945864486]
15 [12.511409444853452, 8.756648834533696, 1.8983128196684138]
16 [2

In [5]:
entt=[]
for i in range (0,128):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)



四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.

四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [  0. 100.   0.]
{90.0, 50.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [100.   0. 100.]
{80.0, 40.0, 50.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416

In [6]:
import heapq #获取list中最小的

f=int(len(entt)*0.6) #计算滤波器熵个数的80%
m=entt
max_number=heapq.nsmallest(f,m) #从m中找出最小的f个数，最大用nlargest
max_index=[]
for t in max_number:
    index=m.index(t)
    max_index.append(index)
    m[index]=float('-inf')
print(max_number)#输出最小的f个数
print(max_index)#输出对应索引

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896,

In [7]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_4 = delete_channels(model, model.layers[13],max_index)
model_4.summary()
model_4.save('8.7.6ent_b_cifar10vgg16_4.h5')
model_4.save_weights('8.7.6ent_b_cifar10vgg16__weights.h5',overwrite=True)

Deleting 76/128 channels from layer: conv2d_4
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (

In [8]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_4.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_4.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_4.save('8.6.5ent_b_cifar10vgg16_4_1.h5')
model_4.save_weights('8.6.5ent_b_cifar10vgg16_4_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 30s 659us/step - loss: 0.8101 - acc: 0.8225 - val_loss: 0.8679 - val_acc: 0.8022
Epoch 2/200
45000/45000 [==============================] - 22s 484us/step - loss: 0.7165 - acc: 0.8538 - val_loss: 0.7579 - val_acc: 0.8420
Epoch 3/200
45000/45000 [==============================] - 22s 481us/step - loss: 0.7070 - acc: 0.8585 - val_loss: 0.7092 - val_acc: 0.8584cc: 0.858
Epoch 4/200
45000/45000 [==============================] - 22s 479us/step - loss: 0.6880 - acc: 0.8653 - val_loss: 0.7699 - val_acc: 0.8412
Epoch 5/200
45000/45000 [==============================] - 22s 482us/step - loss: 0.6777 - acc: 0.8693 - val_loss: 0.7166 - val_acc: 0.8622
Epoch 6/200
45000/45000 [==============================] - 21s 475us/step - loss: 0.6809 - acc: 0.8718 - val_loss: 0.7268 - val_acc: 0.8540
Epoch 7/200
45000/45000 [==============================] - 21s 476us/step - loss: 0.6795 - acc: 0.8710

45000/45000 [==============================] - 22s 483us/step - loss: 0.6360 - acc: 0.8894 - val_loss: 0.7273 - val_acc: 0.8684
Epoch 58/200
45000/45000 [==============================] - 22s 483us/step - loss: 0.6322 - acc: 0.8911 - val_loss: 0.7653 - val_acc: 0.8546
Epoch 59/200
45000/45000 [==============================] - 22s 481us/step - loss: 0.6410 - acc: 0.8878 - val_loss: 0.7665 - val_acc: 0.8540
Epoch 60/200
45000/45000 [==============================] - 22s 481us/step - loss: 0.6264 - acc: 0.8918 - val_loss: 0.7229 - val_acc: 0.8632
Epoch 61/200
45000/45000 [==============================] - 22s 481us/step - loss: 0.6352 - acc: 0.8890 - val_loss: 0.9751 - val_acc: 0.7998
Epoch 62/200
45000/45000 [==============================] - 22s 482us/step - loss: 0.6422 - acc: 0.8874 - val_loss: 0.7872 - val_acc: 0.8548
Epoch 63/200
45000/45000 [==============================] - 22s 482us/step - loss: 0.6362 - acc: 0.8895 - val_loss: 0.7783 - val_acc: 0.8498
Epoch 64/200
45000/45000 [

45000/45000 [==============================] - 22s 481us/step - loss: 0.6318 - acc: 0.8911 - val_loss: 0.7925 - val_acc: 0.8430
Epoch 116/200
45000/45000 [==============================] - 22s 480us/step - loss: 0.6301 - acc: 0.8912 - val_loss: 0.7547 - val_acc: 0.8604
Epoch 117/200
45000/45000 [==============================] - 22s 478us/step - loss: 0.6270 - acc: 0.8922 - val_loss: 0.7443 - val_acc: 0.8628
Epoch 118/200
45000/45000 [==============================] - 22s 483us/step - loss: 0.6131 - acc: 0.8954 - val_loss: 0.9722 - val_acc: 0.8024
Epoch 119/200
45000/45000 [==============================] - 22s 482us/step - loss: 0.6274 - acc: 0.8926 - val_loss: 0.8127 - val_acc: 0.8362
Epoch 120/200
45000/45000 [==============================] - 22s 484us/step - loss: 0.6215 - acc: 0.8943 - val_loss: 0.7341 - val_acc: 0.8638
Epoch 121/200
45000/45000 [==============================] - 22s 482us/step - loss: 0.6340 - acc: 0.8900 - val_loss: 0.7608 - val_acc: 0.8604
Epoch 122/200
45000/

45000/45000 [==============================] - 22s 485us/step - loss: 0.6099 - acc: 0.8975 - val_loss: 0.7266 - val_acc: 0.8710
Epoch 173/200
45000/45000 [==============================] - 22s 482us/step - loss: 0.6115 - acc: 0.8963 - val_loss: 0.7046 - val_acc: 0.8684
Epoch 174/200
45000/45000 [==============================] - 22s 481us/step - loss: 0.6076 - acc: 0.8988 - val_loss: 0.7320 - val_acc: 0.8632
Epoch 175/200
45000/45000 [==============================] - 21s 477us/step - loss: 0.6115 - acc: 0.8965 - val_loss: 0.7636 - val_acc: 0.8580
Epoch 176/200
45000/45000 [==============================] - 22s 483us/step - loss: 0.6187 - acc: 0.8942 - val_loss: 0.7574 - val_acc: 0.8566
Epoch 177/200
45000/45000 [==============================] - 22s 480us/step - loss: 0.6129 - acc: 0.8958 - val_loss: 0.7386 - val_acc: 0.86460.89
Epoch 178/200
45000/45000 [==============================] - 22s 480us/step - loss: 0.6109 - acc: 0.8966 - val_loss: 0.9504 - val_acc: 0.8018
Epoch 179/200
45

In [9]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_4.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_4.evaluate(x_test,y_test,verbose=0)

[0.7926940968513488, 0.8423, 0.99, 0.8423]

In [10]:
score=model_4.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：0.7927
测试准确率为：0.8423


In [11]:
tensor =[]
for i in range(0,128):
   
    u=model_4.get_layer('conv2d_3').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [18.991333289886825, 8.577113524457863, 22.13858709870182]
1 [5.166963440565113, 19.464758927728685, 3.6774239674999625]
2 [5.06771346664038, 11.127846557253116, 13.810227496284973]
3 [27.32610786893458, 25.88465084936154, 17.519336086108993]
4 [10.090865535001614, 41.40366930710713, 9.85733170654223]
5 [28.55270010611272, 11.739058330793592, 23.135302990927965]
6 [7.654492951237571, 3.1211834299048804, 17.911182822589286]
7 [12.40541721498812, 22.755171900573284, 4.5438278842217015]
8 [7.611988519607803, 11.359744835963072, 12.893112737871805]
9 [3.784221899966173, 58.85938037925961, 34.47135388386168]
10 [28.294144248532948, 15.879104924571989, 22.071218437573666]
11 [23.246606394532186, 12.408400726306139, 7.409383569629593]
12 [10.419049613502798, 17.696129624655978, 17.70347332253998]
13 [5.664887403731101, 13.033574269808438, 10.904106158967013]
14 [15.171697676434626, 16.896566695710703, 11.296192027191532]
15 [24.223377519095415, 143.74741718128038, 55.057022636328135]
16 [9.

In [12]:
entt=[]
for i in range (0,128):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个

In [13]:
import heapq #获取list中最小的

f=int(len(entt)*0.6) #计算滤波器熵个数的80%
m=entt
max_number=heapq.nsmallest(f,m) #从m中找出最小的f个数，最大用nlargest
max_index=[]
for t in max_number:
    index=m.index(t)
    max_index.append(index)
    m[index]=float('-inf')
print(max_number)#输出最小的f个数
print(max_index)#输出对应索引

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.91

In [14]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_3 = delete_channels(model_4, model_4.layers[9],max_index)
model_3.summary()
model_3.save('8.6.5ent_b_cifar10vgg16_3.h5')
model_3.save_weights('8.6.5ent_b_cifar10vgg16_3_weights.h5',overwrite=True)

Deleting 76/128 channels from layer: conv2d_3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (

In [15]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_3.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_3.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_3.save('8.6.5ent_b_cifar10vgg16_3_1.h5')
model_3.save_weights('8.6.5ent_b_cifar10vgg16_3_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 27s 595us/step - loss: 0.7602 - acc: 0.8333 - val_loss: 0.7117 - val_acc: 0.8520
Epoch 2/200
45000/45000 [==============================] - 21s 463us/step - loss: 0.7089 - acc: 0.8493 - val_loss: 0.6782 - val_acc: 0.8648
Epoch 3/200
45000/45000 [==============================] - 21s 464us/step - loss: 0.6970 - acc: 0.8552 - val_loss: 0.7281 - val_acc: 0.8454
Epoch 4/200
45000/45000 [==============================] - 21s 461us/step - loss: 0.6922 - acc: 0.8576 - val_loss: 0.7328 - val_acc: 0.8524
Epoch 5/200
45000/45000 [==============================] - 21s 458us/step - loss: 0.6953 - acc: 0.8590 - val_loss: 0.7196 - val_acc: 0.8546
Epoch 6/200
45000/45000 [==============================] - 21s 460us/step - loss: 0.6938 - acc: 0.8603 - val_loss: 0.7257 - val_acc: 0.8500
Epoch 7/200
45000/45000 [==============================] - 21s 463us/step - loss: 0.6940 - acc: 0.8609 - val_lo

45000/45000 [==============================] - 21s 460us/step - loss: 0.6538 - acc: 0.8756 - val_loss: 0.7287 - val_acc: 0.8528
Epoch 59/200
45000/45000 [==============================] - 21s 464us/step - loss: 0.6550 - acc: 0.8761 - val_loss: 0.7170 - val_acc: 0.8620
Epoch 60/200
45000/45000 [==============================] - 21s 462us/step - loss: 0.6572 - acc: 0.8750 - val_loss: 0.7009 - val_acc: 0.8672
Epoch 61/200
45000/45000 [==============================] - 21s 461us/step - loss: 0.6544 - acc: 0.8768 - val_loss: 0.8994 - val_acc: 0.7994
Epoch 62/200
45000/45000 [==============================] - 21s 456us/step - loss: 0.6539 - acc: 0.8773 - val_loss: 0.8059 - val_acc: 0.8314
Epoch 63/200
45000/45000 [==============================] - 21s 459us/step - loss: 0.6580 - acc: 0.8738 - val_loss: 0.7934 - val_acc: 0.8350
Epoch 64/200
45000/45000 [==============================] - 21s 462us/step - loss: 0.6563 - acc: 0.8749 - val_loss: 0.6851 - val_acc: 0.8740
Epoch 65/200
45000/45000 [

45000/45000 [==============================] - 21s 462us/step - loss: 0.6526 - acc: 0.8786 - val_loss: 0.7601 - val_acc: 0.8476
Epoch 117/200
45000/45000 [==============================] - 21s 462us/step - loss: 0.6523 - acc: 0.8766 - val_loss: 0.7454 - val_acc: 0.8536
Epoch 118/200
45000/45000 [==============================] - 21s 457us/step - loss: 0.6633 - acc: 0.8731 - val_loss: 0.6961 - val_acc: 0.8650
Epoch 119/200
45000/45000 [==============================] - 21s 461us/step - loss: 0.6493 - acc: 0.8772 - val_loss: 0.7534 - val_acc: 0.8480
Epoch 120/200
45000/45000 [==============================] - 21s 462us/step - loss: 0.6472 - acc: 0.8774 - val_loss: 0.7344 - val_acc: 0.8532
Epoch 121/200
45000/45000 [==============================] - 21s 463us/step - loss: 0.6484 - acc: 0.8801 - val_loss: 0.7971 - val_acc: 0.8366
Epoch 122/200
45000/45000 [==============================] - 21s 460us/step - loss: 0.6487 - acc: 0.8785 - val_loss: 0.7387 - val_acc: 0.8524
Epoch 123/200
45000/

45000/45000 [==============================] - 21s 458us/step - loss: 0.6411 - acc: 0.8790 - val_loss: 0.6814 - val_acc: 0.8712
Epoch 174/200
45000/45000 [==============================] - 21s 460us/step - loss: 0.6476 - acc: 0.8770 - val_loss: 0.7911 - val_acc: 0.8378
Epoch 175/200
45000/45000 [==============================] - 21s 458us/step - loss: 0.6473 - acc: 0.8782 - val_loss: 0.7455 - val_acc: 0.8532
Epoch 176/200
45000/45000 [==============================] - 21s 460us/step - loss: 0.6442 - acc: 0.8800 - val_loss: 0.7157 - val_acc: 0.8590
Epoch 177/200
45000/45000 [==============================] - 21s 459us/step - loss: 0.6640 - acc: 0.8734 - val_loss: 0.7010 - val_acc: 0.8648
Epoch 178/200
45000/45000 [==============================] - 21s 462us/step - loss: 0.6403 - acc: 0.8800 - val_loss: 0.6808 - val_acc: 0.8698
Epoch 179/200
45000/45000 [==============================] - 21s 459us/step - loss: 0.6450 - acc: 0.8774 - val_loss: 0.8293 - val_acc: 0.8174
Epoch 180/200
45000/

In [16]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_3.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_3.evaluate(x_test,y_test,verbose=0)

[0.7084896286964416, 0.8608, 0.9932, 0.8608]

In [17]:
score=model_3.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：0.7085
测试准确率为：0.8608


In [18]:
tensor =[]
for i in range(0,64):
   
    u=model_3.get_layer('conv2d_2').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [14.401949706143517, 7.5589079246974125, 11.477813268219702]
1 [4.760339469895873, 18.547938508391486, 6.635745277432136]
2 [2.378491934039505, 13.95086877230064, 13.300709325116998]
3 [11.55166605042967, 18.82176036340624, 10.216877570368691]
4 [14.791874629784004, 17.64735468041426, 26.430024431580343]
5 [13.16990652342276, 8.465297994849417, 14.28368535552048]
6 [10.528161430805387, 14.590282984075031, 24.651073342684516]
7 [31.671242721261084, 19.422211868912367, 27.847662996485806]
8 [1.6664256477925956, 17.982417261067752, 14.614368193337326]
9 [9.887562788463747, 5.535306057622315, 2.709904102726087]
10 [18.0144536612761, 5.541357256442709, 1.9509795956192322]
11 [17.70383715228286, 18.547048129243414, 28.82134289862996]
12 [11.031415589578348, 4.935862185056673, 6.369193780204056]
13 [52.43725160305766, 115.64538249401703, 119.37822050374034]
14 [11.415896624099076, 5.044313897018146, 15.49179044321458]
15 [6.910286789475681, 13.98552589156021, 12.96335289116389]
16 [30.67333

In [19]:
entt=[]
for i in range (0,64):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 

In [20]:
import heapq #获取list中最小的

f=int(len(entt)*0.5) #计算滤波器熵个数的80%
m=entt
max_number=heapq.nsmallest(f,m) #从m中找出最小的f个数，最大用nlargest
max_index=[]
for t in max_number:
    index=m.index(t)
    max_index.append(index)
    m[index]=float('-inf')
print(max_number)#输出最小的f个数
print(max_index)#输出对应索引

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896]
[0, 5, 15, 18, 19, 21, 23, 25, 27, 30, 31, 38, 44, 46, 47, 51, 53, 58, 61, 63, 2, 3, 6, 7, 9, 11, 12, 13, 14, 16, 17, 20]


In [21]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_2 = delete_channels(model_3, model_3.layers[5],max_index)
model_2.summary()
model_2.save('8.6.5ent_b_cifar10vgg16_2.h5')
model_2.save_weights('8.6.5ent_b_cifar10vgg16_2_weights.h5',overwrite=True)

Deleting 32/64 channels from layer: conv2d_2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        18464     
_________________________________________________________________
activation_2 (Activation)    mu

In [22]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_2.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_2.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_2.save('8.6.5ent_b_cifar10vgg16_2_1.h5')
model_2.save_weights('8.6.5ent_b_cifar10vgg16_2_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 27s 589us/step - loss: 0.7773 - acc: 0.8274 - val_loss: 0.7075 - val_acc: 0.8522
Epoch 2/200
45000/45000 [==============================] - 20s 435us/step - loss: 0.7269 - acc: 0.8441 - val_loss: 0.8982 - val_acc: 0.7984
Epoch 3/200
45000/45000 [==============================] - 20s 436us/step - loss: 0.7169 - acc: 0.8495 - val_loss: 0.7336 - val_acc: 0.8422
Epoch 4/200
45000/45000 [==============================] - 20s 435us/step - loss: 0.7102 - acc: 0.8514 - val_loss: 0.6971 - val_acc: 0.8576
Epoch 5/200
45000/45000 [==============================] - 19s 432us/step - loss: 0.7012 - acc: 0.8560 - val_loss: 0.7771 - val_acc: 0.8352
Epoch 6/200
45000/45000 [==============================] - 19s 429us/step - loss: 0.6987 - acc: 0.8571 - val_loss: 0.7880 - val_acc: 0.8342
Epoch 7/200
45000/45000 [==============================] - 19s 429us/step - loss: 0.7028 - acc: 0.8569 - val_lo

Epoch 59/200
45000/45000 [==============================] - 19s 432us/step - loss: 0.6833 - acc: 0.8658 - val_loss: 0.7057 - val_acc: 0.8588
Epoch 60/200
45000/45000 [==============================] - 19s 432us/step - loss: 0.6704 - acc: 0.8687 - val_loss: 0.7080 - val_acc: 0.8564
Epoch 61/200
45000/45000 [==============================] - 20s 434us/step - loss: 0.6738 - acc: 0.8680 - val_loss: 0.7712 - val_acc: 0.8428
Epoch 62/200
45000/45000 [==============================] - 20s 435us/step - loss: 0.6747 - acc: 0.8676 - val_loss: 0.7808 - val_acc: 0.8458
Epoch 63/200
45000/45000 [==============================] - 20s 435us/step - loss: 0.6665 - acc: 0.8702 - val_loss: 0.7218 - val_acc: 0.8602
Epoch 64/200
45000/45000 [==============================] - 20s 437us/step - loss: 0.6604 - acc: 0.8715 - val_loss: 0.7169 - val_acc: 0.8586
Epoch 65/200
45000/45000 [==============================] - 20s 434us/step - loss: 0.6722 - acc: 0.8674 - val_loss: 0.8187 - val_acc: 0.8260
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 19s 433us/step - loss: 0.6674 - acc: 0.8699 - val_loss: 0.7333 - val_acc: 0.8534
Epoch 118/200
45000/45000 [==============================] - 20s 437us/step - loss: 0.6642 - acc: 0.8699 - val_loss: 0.7114 - val_acc: 0.8622
Epoch 119/200
45000/45000 [==============================] - 19s 430us/step - loss: 0.6598 - acc: 0.8723 - val_loss: 0.6994 - val_acc: 0.8648
Epoch 120/200
45000/45000 [==============================] - 20s 434us/step - loss: 0.6647 - acc: 0.8689 - val_loss: 0.7486 - val_acc: 0.8474
Epoch 121/200
45000/45000 [==============================] - 20s 437us/step - loss: 0.6514 - acc: 0.8735 - val_loss: 0.7088 - val_acc: 0.8632
Epoch 122/200
45000/45000 [==============================] - 20s 436us/step - loss: 0.6582 - acc: 0.8716 - val_loss: 0.7353 - val_acc: 0.8478
Epoch 123/200
45000/45000 [==============================] - 19s 433us/step - loss: 0.6610 - acc: 0.8719 - val_loss: 0.7184 - val_acc: 0.8560
Epoch 

Epoch 175/200
45000/45000 [==============================] - 20s 435us/step - loss: 0.6546 - acc: 0.8736 - val_loss: 0.7631 - val_acc: 0.8402
Epoch 176/200
45000/45000 [==============================] - 20s 436us/step - loss: 0.6445 - acc: 0.8771 - val_loss: 0.6989 - val_acc: 0.8580
Epoch 177/200
45000/45000 [==============================] - 20s 438us/step - loss: 0.6485 - acc: 0.8755 - val_loss: 0.9518 - val_acc: 0.7840
Epoch 178/200
45000/45000 [==============================] - 20s 437us/step - loss: 0.6496 - acc: 0.8749 - val_loss: 0.9783 - val_acc: 0.7868
Epoch 179/200
45000/45000 [==============================] - 20s 439us/step - loss: 0.6556 - acc: 0.8736 - val_loss: 0.7413 - val_acc: 0.8462
Epoch 180/200
45000/45000 [==============================] - 20s 433us/step - loss: 0.6481 - acc: 0.8734 - val_loss: 0.7145 - val_acc: 0.8596
Epoch 181/200
45000/45000 [==============================] - 20s 438us/step - loss: 0.6633 - acc: 0.8700 - val_loss: 0.7065 - val_acc: 0.8600
Epoch 

In [23]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_2.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_2.evaluate(x_test,y_test,verbose=0)

[0.8455086454391479, 0.8194, 0.9902, 0.8194]

In [24]:
score=model_2.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：0.8455
测试准确率为：0.8194


In [25]:
tensor =[]
for i in range(0,64):
   
    u=model_2.get_layer('conv2d_1').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [2.0450879396633783, 2.1083841327784483, 1.285267694621184]
1 [2.9154759474226504, 1.3228756555322954, 0.5590169943749475]
2 [2.017294297375586, 4.121371609690142, 5.446591844054512]
3 [2.5369909678440226, 10.207552639958413, 8.831261420326921]
4 [0.9882006428568496, 1.4583378766564883, 0.8561259161315505]
5 [3.4348286565999024, 1.716220421986735, 1.2730988596356685]
6 [2.843714392715533, 2.6701209706356464, 11.063107775571373]
7 [2.227754615077702, 2.4980461114639176, 1.653594569415369]
8 [10.110114210598656, 10.218423295704296, 2.001953065453505]
9 [12.611061947657188, 2.0000011813576455, 1.2348743471297698]
10 [2.099106952968333, 1.9921721813136535, 2.692582403567252]
11 [2.413712040915332, 9.809795340980354, 2.571836848572772]
12 [30.66874045788391, 21.8770303056471, 9.106411905099579]
13 [13.856441699170102, 0.0, 11.215700207848984]
14 [8.51305487808666, 11.551561439541976, 2.1610195494446764]
15 [1.557491974297139, 1.25, 2.0223362109204297]
16 [1.8192224472284855, 1.50779226022

In [26]:
entt=[]
for i in range (0,64):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416

In [27]:
import heapq #获取list中最小的

f=int(len(entt)*0.5) #计算滤波器熵个数的80%
m=entt
max_number=heapq.nsmallest(f,m) #从m中找出最小的f个数，最大用nlargest
max_index=[]
for t in max_number:
    index=m.index(t)
    max_index.append(index)
    m[index]=float('-inf')
print(max_number)#输出最小的f个数
print(max_index)#输出对应索引

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9182958340544896]
[0, 1, 4, 5, 7, 10, 15, 16, 17, 18, 19, 23, 24, 28, 29, 30, 32, 34, 35, 40, 43, 46, 47, 48, 53, 54, 56, 57, 58, 62, 63, 2]


In [28]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_1 = delete_channels(model_2, model_2.layers[1],max_index)
model_1.summary()
model_1.save('8.6.5ent_b_cifar10vgg16_1.h5')
model_1.save_weights('8.6.5ent_b_cifar10vgg16_1_weights.h5',overwrite=True)

Deleting 32/64 channels from layer: conv2d_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    mu

In [29]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_1.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_1.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_1.save('8.6.5ent_b_cifar10vgg16_1_1.h5')
model_1.save_weights('8.6.5ent_b_cifar10vgg16_1_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 23s 519us/step - loss: 0.7201 - acc: 0.8478 - val_loss: 0.8940 - val_acc: 0.8000
Epoch 2/200
45000/45000 [==============================] - 15s 338us/step - loss: 0.6867 - acc: 0.8592 - val_loss: 0.7272 - val_acc: 0.8502
Epoch 3/200
45000/45000 [==============================] - 15s 341us/step - loss: 0.6976 - acc: 0.8559 - val_loss: 0.8300 - val_acc: 0.8192
Epoch 4/200
45000/45000 [==============================] - 15s 337us/step - loss: 0.6889 - acc: 0.8601 - val_loss: 0.7205 - val_acc: 0.8552
Epoch 5/200
45000/45000 [==============================] - 15s 337us/step - loss: 0.6915 - acc: 0.8589 - val_loss: 0.8627 - val_acc: 0.8058
Epoch 6/200
45000/45000 [==============================] - 15s 336us/step - loss: 0.6838 - acc: 0.8608 - val_loss: 0.7104 - val_acc: 0.8554
Epoch 7/200
45000/45000 [==============================] - 15s 335us/step - loss: 0.6819 - acc: 0.8639 - val_lo

45000/45000 [==============================] - 15s 340us/step - loss: 0.6744 - acc: 0.8683 - val_loss: 0.7503 - val_acc: 0.8488
Epoch 59/200
45000/45000 [==============================] - 15s 337us/step - loss: 0.6722 - acc: 0.8672 - val_loss: 0.7113 - val_acc: 0.8590
Epoch 60/200
45000/45000 [==============================] - 15s 336us/step - loss: 0.6724 - acc: 0.8676 - val_loss: 0.7849 - val_acc: 0.8378
Epoch 61/200
45000/45000 [==============================] - 15s 338us/step - loss: 0.6781 - acc: 0.8664 - val_loss: 0.7092 - val_acc: 0.8598
Epoch 62/200
45000/45000 [==============================] - 15s 339us/step - loss: 0.6703 - acc: 0.8678 - val_loss: 0.7434 - val_acc: 0.8460
Epoch 63/200
45000/45000 [==============================] - 15s 338us/step - loss: 0.6712 - acc: 0.8667 - val_loss: 0.7248 - val_acc: 0.8506
Epoch 64/200
45000/45000 [==============================] - 15s 337us/step - loss: 0.6655 - acc: 0.8696 - val_loss: 0.7252 - val_acc: 0.8578
Epoch 65/200
45000/45000 [

45000/45000 [==============================] - 15s 335us/step - loss: 0.6596 - acc: 0.8702 - val_loss: 0.7622 - val_acc: 0.8408
Epoch 117/200
45000/45000 [==============================] - 15s 334us/step - loss: 0.6652 - acc: 0.8710 - val_loss: 0.7327 - val_acc: 0.8504
Epoch 118/200
45000/45000 [==============================] - 15s 336us/step - loss: 0.6554 - acc: 0.8740 - val_loss: 0.7271 - val_acc: 0.8492
Epoch 119/200
45000/45000 [==============================] - 15s 336us/step - loss: 0.6667 - acc: 0.8681 - val_loss: 0.7103 - val_acc: 0.8532
Epoch 120/200
45000/45000 [==============================] - 15s 336us/step - loss: 0.6678 - acc: 0.8683 - val_loss: 0.7585 - val_acc: 0.8458
Epoch 121/200
45000/45000 [==============================] - 15s 337us/step - loss: 0.6575 - acc: 0.8714 - val_loss: 0.6853 - val_acc: 0.8680
Epoch 122/200
45000/45000 [==============================] - 15s 334us/step - loss: 0.6635 - acc: 0.8711 - val_loss: 0.6983 - val_acc: 0.8622
Epoch 123/200
45000/

Epoch 174/200
45000/45000 [==============================] - 15s 336us/step - loss: 0.6540 - acc: 0.8723 - val_loss: 0.7344 - val_acc: 0.8558
Epoch 175/200
45000/45000 [==============================] - 15s 334us/step - loss: 0.6569 - acc: 0.8717 - val_loss: 0.7021 - val_acc: 0.8564
Epoch 176/200
45000/45000 [==============================] - 15s 335us/step - loss: 0.6536 - acc: 0.8737 - val_loss: 0.7379 - val_acc: 0.8454
Epoch 177/200
45000/45000 [==============================] - 15s 336us/step - loss: 0.6598 - acc: 0.8696 - val_loss: 0.7325 - val_acc: 0.8530
Epoch 178/200
45000/45000 [==============================] - 15s 335us/step - loss: 0.6631 - acc: 0.8693 - val_loss: 0.7773 - val_acc: 0.8378
Epoch 179/200
45000/45000 [==============================] - 15s 336us/step - loss: 0.6597 - acc: 0.8708 - val_loss: 0.7387 - val_acc: 0.8490
Epoch 180/200
45000/45000 [==============================] - 15s 334us/step - loss: 0.6583 - acc: 0.8710 - val_loss: 0.7160 - val_acc: 0.8520
Epoch 

In [30]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_1.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_1.evaluate(x_test,y_test,verbose=0)

[0.7962649865150452, 0.8339, 0.9908, 0.8339]

In [31]:
score=model_1.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：0.7963
测试准确率为：0.8339
